In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd

import requests
def fetch_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return f"Error: Unable to fetch the HTML file. Status code: {response.status_code}"
    
url = "http://sagarin.com/sports/cfsend.htm"
html_content = fetch_html(url)
print(type(html_content))

<class 'str'>


In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_content,'html.parser')

In [3]:
content = soup.find_all('pre')[2].string

In [4]:
content_list = content.splitlines()

In [5]:
import re
def getSagarinData(inputString):
    pattern = r'\s*(\d+)\s+([A-Za-z\s\(\)\-&.\']+)\s+([A-Za-z]+)\s+=\s+([\d.]+)\s+(\d+)\s+(\d+)\s+([\d.]+)\(\s*(\d+)\)\s+(\d+)\s+(\d+)\s+\|\s+(\d+)\s+(\d+)\s+\|\s+([\d.]+)\s+(\d+)\s+\|\s+([\d.]+)\s+(\d+)\s+\|\s+([\d.]+)\s+(\d+)\s+\|\s+([\d.]+)\s+(\d+)\s+([A-Z\s\d.\-/=>]+)\s+\(([A-Za-z]+)\)'
    matches = re.findall(pattern,inputString)
    try:
        parsed_list = list(matches[0])
    except:
        return [0,'N/A',0]
    output = [int(parsed_list[0]), parsed_list[1].rstrip(), float(parsed_list[3])]
    return output

In [6]:
test_list = content_list[190]
test_list

' 144  Villanova            AA =  53.02    8   3   46.76( 193)    0   0  |    0   0  |   51.22  149 |   54.85  137 |   49.78  144 |   50.35  146  COASTAL             (AA)'

In [7]:
data = []
for line in content_list:
    newdata = getSagarinData(line)
    if newdata != [0, 'N/A', 0]:
        data.append(getSagarinData(line))
df = pd.DataFrame(data)
df.columns = ['Rank','Team','Power Rating']
df.head()

,Rank,Team,Power Rating
0,1,Alabama,92.37
1,2,Texas,92.33
2,3,Ohio State,92.11
3,4,Georgia,92.07
4,5,Mississippi,91.96


In [8]:
df.describe()

,Rank,Power Rating
count,263.000000,263.000000
mean,132.000000,57.241103
std,76.065761,16.495336
min,1.000000,21.800000
25%,66.500000,44.100000
50%,132.000000,55.570000
75%,197.500000,71.500000
max,263.000000,92.370000


In [9]:
df.to_csv('sagarin.csv')